In [40]:
import pandas as pd

xls_file = pd.read_excel('srd2014\RUNDATA.xls')

df = pd.DataFrame(xls_file)

Example of filtering for a particular inscription

In [41]:
df[df['Signum'].str.contains('Ög 154')]

sigs = df['Signum'].values

In order to create tables with rundata_ids, we have to separate the ID from the runic text in each of the RUNDATA files

In [33]:
def get_table_from_text(file_name):
    with open("srd2014\{}".format(file_name), "r", encoding='iso-8859-1') as f:
        lines = f.readlines()

    df = pd.DataFrame(columns=['Signum', 'Text'])

    for line in lines:
        for signum in sigs:
            if line.find(signum) != -1:
                parts = line.split(signum)
                df = df.append({'Signum': signum, 'Text': parts[1]}, ignore_index=True)

    return df
        
runtext_df = get_table_from_text('RUNTEXT')
english_df = get_table_from_text('ENGLISH')


Now that we have our separated texts, lets insert them into database tables accordingly

In [39]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="rundata",
    user="user",
    password="pass"
)

cur = conn.cursor()


def create_core_table():
    cur.execute("DROP TABLE IF EXISTS inscription")
    cur.execute("""
        CREATE TABLE inscription (
            rundata_id text primary key,
            coordinates text ,
            material text
    )
    """)

    for i, row in df.iterrows():
        cur.execute("INSERT INTO inscription (rundata_id, coordinates, material) VALUES(%s, %s, %s)", (row['Signum'], row['Koordinater'], row['Material']))
    conn.commit()

def create_rundata_table(name, df):
    cur.execute("DROP TABLE IF EXISTS rundata_{}".format(name))
    cur.execute("""
        CREATE TABLE rundata_{} (
            rundata_id text,
            text text
    )
    """.format(name))

    for i, row in df.iterrows():
        cur.execute("INSERT INTO rundata_{} (rundata_id, text) VALUES(%s, %s)".format(name), (row['Signum'], row['Text']))
    conn.commit()

create_core_table()
create_rundata_table('english', english_df)
create_rundata_table('runtext', runtext_df)

cur.close()
conn.close()